In [3]:
import os, json, shutil
from collections import defaultdict
from tqdm import tqdm
import numpy as np

# ===== 경로/입력 =====
base_dir = r"E:\ai05-level1-project"
img_dir  = os.path.join(base_dir, "train_images")

with open(os.path.join(base_dir, "train_master_annotations.json"), "r", encoding="utf-8") as f:
    master_data = json.load(f)
with open(os.path.join(base_dir, "class_to_id.json"), "r", encoding="utf-8") as f:
    class_to_id = json.load(f)

# ===== (A) OOB로 제거할 파일 (이미 점검했던 2개 예시) =====
oob_error_files = {
    "K-003351-016262-018357_0_2_0_2_75_000_200.png",
    "K-003544-004543-012247-016551_0_2_0_2_70_000_200.png",
}

# ===== (B) IoU로 겹침 높은 이미지 자동 탐지 =====
def iou_xyxy(a, b):
    # a,b: [x1,y1,x2,y2]
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    if x2 <= x1 or y2 <= y1:
        return 0.0
    inter = (x2-x1)*(y2-y1)
    area_a = (a[2]-a[0])*(a[3]-a[1])
    area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-9)

def xywh_to_xyxy(box):
    x,y,w,h = box
    return [x, y, x+w, y+h]

HIGH_IOU_TH = 0.5  # 겹침으로 간주할 임계값(보수적으로 설정)
high_iou_files = set()

for fname, info in tqdm(master_data.items(), desc="Scan IoU"):
    boxes = [xywh_to_xyxy(ann['bbox']) for ann in info['annotations']]
    if len(boxes) < 2:
        continue
    bad = False
    for i in range(len(boxes)):
        for j in range(i+1, len(boxes)):
            if iou_xyxy(boxes[i], boxes[j]) >= HIGH_IOU_TH:
                bad = True; break
        if bad: break
    if bad:
        high_iou_files.add(fname)

print(f"OOB files: {len(oob_error_files)} | High-IoU files: {len(high_iou_files)}")

# ===== 최종 제외 목록 =====
bad_files = oob_error_files.union(high_iou_files)
print(f"총 제외 이미지 수: {len(bad_files)}")


Scan IoU: 100%|██████████| 1489/1489 [00:00<00:00, 297849.99it/s]

OOB files: 2 | High-IoU files: 5
총 제외 이미지 수: 7


In [6]:
import random
random.seed(42)

# ===== 출력 폴더 =====
out_root = os.path.join(base_dir, "yolo_baseline")
img_out_train = os.path.join(out_root, "images", "train")
img_out_val   = os.path.join(out_root, "images", "val")
lab_out_train = os.path.join(out_root, "labels", "train")
lab_out_val   = os.path.join(out_root, "labels", "val")
for d in [img_out_train, img_out_val, lab_out_train, lab_out_val]:
    os.makedirs(d, exist_ok=True)

# ===== 유효 샘플 목록 (나쁜 파일 제외) =====
valid_items = [ (fname,info) for fname,info in master_data.items() if fname not in bad_files ]

# ===== 간단 분할 (8:2) =====
random.shuffle(valid_items)
split = int(len(valid_items)*0.8)
train_items = valid_items[:split]
val_items   = valid_items[split:]
print(f"Final split -> train: {len(train_items)}, val: {len(val_items)}")

def coco_to_yolo_line(box_xywh, cls_id, W, H):
    x,y,w,h = box_xywh
    # x,y는 좌상단 -> 중심으로
    xc = x + w/2
    yc = y + h/2
    # 정규화
    return f"{cls_id} {xc/W:.6f} {yc/H:.6f} {w/W:.6f} {h/H:.6f}"

def export_split(items, img_out_dir, lab_out_dir):
    kept, skipped = 0, 0
    for fname, info in tqdm(items, desc=f"Export to {os.path.basename(img_out_dir)}"):
        src_img = info['image_path']
        if not os.path.exists(src_img):
            skipped += 1; continue

        # 라벨 라인 생성
        W, H = info['width'], info['height']
        lines = []
        for ann in info['annotations']:
            x, y, w, h = ann['bbox']
            # 유효성 가볍게 한번 더 체크
            if w <= 1 or h <= 1:     # 너무 작은 박스 제거
                continue
            # 이미지 경계 벗어나도 YOLO가 클립하지만, 음수 등은 제거
            if x < 0 or y < 0 or x+w > W or y+h > H:
                continue
            cls_id = ann['class_id']
            lines.append(coco_to_yolo_line([x,y,w,h], cls_id, W, H))

        if len(lines) == 0:
            # 학습에 쓸 라벨이 없으면 스킵(필요시 빈 라벨로도 가능)
            skipped += 1; continue

        # 이미지/라벨 복사 & 저장
        dst_img = os.path.join(img_out_dir, fname)
        shutil.copy2(src_img, dst_img)

        label_name = os.path.splitext(fname)[0] + ".txt"
        with open(os.path.join(lab_out_dir, label_name), "w", encoding="utf-8") as f:
            f.write("\n".join(lines))
        kept += 1
    print(f"kept={kept}, skipped(no label or missing img)={skipped}")

export_split(train_items, img_out_train, lab_out_train)
export_split(val_items, img_out_val, lab_out_val)

# ===== data.yaml 생성 =====
names_list = [None] * (max(class_to_id.values())+1)
for name, idx in class_to_id.items():
    names_list[idx] = name

from pathlib import Path
path_str = Path(out_root).as_posix()   # <-- 경로를 / 형태로 변환

data_yaml = f"""# YOLO dataset config
path: {path_str}
train: images/train
val: images/val
nc: {len(names_list)}
names: {names_list}
"""

with open(os.path.join(out_root, "data.yaml"), "w", encoding="utf-8") as f:
    f.write(data_yaml)

print("data.yaml 생성 완료:", os.path.join(out_root, "data.yaml"))


Final split -> train: 1185, val: 297


Export to train: 100%|██████████| 1185/1185 [00:07<00:00, 152.67it/s]


kept=1185, skipped(no label or missing img)=0


Export to val: 100%|██████████| 297/297 [00:01<00:00, 161.33it/s]

kept=297, skipped(no label or missing img)=0
data.yaml 생성 완료: E:\ai05-level1-project\yolo_baseline\data.yaml


In [10]:
import torch
print("torch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)     # 예상: '12.1'
print("cuda available:", torch.cuda.is_available())  # True
print("device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


torch: 2.9.0+cpu
torch.version.cuda: None
cuda available: False
device: None


In [1]:
import sys, torch
print("PY:", sys.executable)
print("TORCH FILE:", getattr(torch, "__file__", "not loaded"))
print("TORCH VER:", torch.__version__)


PY: e:\anaconda3\envs\codeit_project_env\python.exe
TORCH FILE: e:\anaconda3\envs\codeit_project_env\Lib\site-packages\torch\__init__.py
TORCH VER: 2.5.1+cu121


In [2]:
import torch
print("torch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


torch: 2.5.1+cu121
torch.version.cuda: 12.1
cuda available: True
device: NVIDIA GeForce RTX 4090


In [3]:
from ultralytics import YOLO
model = YOLO("yolov8n.yaml")
model.train(data="E:/ai05-level1-project/yolo_baseline/data.yaml",
            epochs=50, imgsz=640, batch=16, device=0)


Ultralytics 8.3.220  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:/ai05-level1-project/yolo_baseline/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48372 (\N{HANGUL SYLLABLE BO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 47161 (\N{HANGUL SYLLABLE RYEONG}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48512 (\N{HANGUL SYLLABLE BU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 49828 (\N{HANGUL SYLLABLE SEU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) Dej

                   all        297        915      0.554      0.768      0.688      0.634
             5mg         38         38      0.549          1      0.854      0.781
          800mg         39         39      0.807          1      0.831      0.755
          500/20mg          6          6      0.663      0.987      0.913      0.822
                            6          6      0.388      0.833      0.558      0.529
            100mg         28         28      0.466      0.997      0.745      0.739
         30mg          7          7      0.584      0.857      0.867      0.777
          ()          6          6       0.55          1      0.705      0.693
                       8          8      0.332          1      0.763      0.738
          500/20mg         13         13      0.936          1      0.995      0.934
                            6          6      0.592      0.971      0.819      0.691
              10mg          8          8          1      0.218      0.483      0.45

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000027B9F4E5FD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026, 